In [41]:
import requests
import pandas as pd
import feedparser
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [98]:
url = 'https://bcs-express.ru/category/torgovye-rekomendacii'
driver = webdriver.Chrome()
driver.get(url)

In [104]:
data = {'title': [], 'href': []}
scroll_pause_time = 2
scroll_height = driver.execute_script("return document.body.scrollHeight")
last_height = 0
button = None

In [105]:
while True:
    driver.execute_script("window.scrollBy(0, 800)")
    time.sleep(scroll_pause_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    titles = soup.find_all('a', class_='iKzE')

    for headline in titles:
        data['title'].append(headline)
        href = headline.get('href')
        data['href'].append(href)
    new_scroll_height = driver.execute_script("return document.body.scrollHeight")
    if new_scroll_height == scroll_height:
        print('Достигнут конец страницы')
        try:
            button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-id='button-more']"))
            )

            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(scroll_pause_time)
            driver.execute_script("arguments[0].click();", button)
            print("Кнопка успешно нажата.")
            time.sleep(scroll_pause_time)
        except:
            print('Заебал листать долбоёб')
            break
        
    scroll_height = new_scroll_height

Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно нажата.
Достигнут конец страницы
Кнопка успешно 

In [109]:
set(data['href'])

{'https://bcs-express.ru/novosti-i-analitika/surgutneftegaz-short-pri-vozvrate-kotirovok-k-35-5',
 'https://bcs-express.ru/novosti-i-analitika/sberbank-seredina-kanala-ne-samoe-udobnoe-mesto-dlia-otkrytiia-pozitsii',
 'https://bcs-express.ru/novosti-i-analitika/torgovaia-rekomendatsiia-riu5-k-pokupkam-zhelatel-no-prismatrivat-sia-lish-v-sluchae-proboia-vverkh-otmetki-86800',
 'https://bcs-express.ru/novosti-i-analitika/mts-eshche-ne-pozdno',
 'https://bcs-express.ru/novosti-i-analitika/kakie-aktsii-luchshe-pokupat-na-otskok',
 'https://bcs-express.ru/novosti-i-analitika/f-iuchers-na-indeks-mmvb-medvedi-mogut-prodavit-tsenu-u-182000',
 'https://bcs-express.ru/novosti-i-analitika/lozhnyi-proboi-3000-v-indekse-mosbirzhi-kuda-mozhem-upast',
 'https://bcs-express.ru/novosti-i-analitika/lukoil-po-prezhnemu-vne-rynka',
 'https://bcs-express.ru/novosti-i-analitika/lukoil-potentsial-dlia-korrektsii-vniz-sokhraniaetsia',
 'https://bcs-express.ru/novosti-i-analitika/investitsii-v-kreditnyi-reitin

In [96]:
set(headlines)

{<a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/aeroflot-klassicheskii-treid-v-diapazone">Аэрофлот. Классический трейд в диапазоне</a>,
 <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/lyondellbasell-industries-mirovoi-lider-v-proizvodstve-polietilena">LyondellBasell Industries — мировой лидер в производстве полиэтилена</a>,
 <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/sberbank-poiavilis-signaly-k-prodazham-no-poka-medvedi-ne-aktivny">Сбербанк: Появились сигналы к продажам, но пока медведи не активны</a>,
 <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/mosbirzha-pokupaem-na-vykhode-iz-kanala">МосБиржа: Покупаем на выходе из канала</a>,
 <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/gazprom-otskok-ot-urovnia-soprotivleniia">ГАЗПРОМ: Отскок от уровня сопротивления</a>,
 <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/polymetal-mozhno-zakryvat-longi">Polymetal: Можно закрывать лон

In [110]:
new_data = {'title': set(data['title']), 'href': set(data['href'])}

In [122]:
new_new_data = {'title': list(new_data['title']), 'href': list(new_data['href'])}

AttributeError: 'list' object has no attribute 'get_text'

In [121]:
new_new_data

{'title': [<a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/aeroflot-klassicheskii-treid-v-diapazone">Аэрофлот. Классический трейд в диапазоне</a>,
  <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/lyondellbasell-industries-mirovoi-lider-v-proizvodstve-polietilena">LyondellBasell Industries — мировой лидер в производстве полиэтилена</a>,
  <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/sberbank-poiavilis-signaly-k-prodazham-no-poka-medvedi-ne-aktivny">Сбербанк: Появились сигналы к продажам, но пока медведи не активны</a>,
  <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/mosbirzha-pokupaem-na-vykhode-iz-kanala">МосБиржа: Покупаем на выходе из канала</a>,
  <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/gazprom-otskok-ot-urovnia-soprotivleniia">ГАЗПРОМ: Отскок от уровня сопротивления</a>,
  <a class="iKzE" href="https://bcs-express.ru/novosti-i-analitika/polymetal-mozhno-zakryvat-longi">Polymetal: Можн

In [130]:
import json
with open('result.json', 'w') as fp:
    json.dump(data, fp)

In [ ]:
titles = []
hrefs = []

In [125]:
for i in range(len(new_new_data['title'])):
    titles.append(new_new_data['title'][i].get_text())

In [128]:
data = {'titles': titles, 'hrefs': new_new_data['href']}

In [129]:
data

{'titles': ['Аэрофлот. Классический трейд в диапазоне',
  'LyondellBasell Industries — мировой лидер в производстве полиэтилена',
  'Сбербанк: Появились сигналы к продажам, но пока медведи не активны',
  'МосБиржа: Покупаем на выходе из канала',
  'ГАЗПРОМ: Отскок от уровня сопротивления',
  'Polymetal: Можно закрывать лонги',
  'Роснефть. Отскок от минимумов дня может получить развитие',
  'МосБиржа: Выход из боковика открывает новые цели',
  'Настроение рынка - RIH6: Участники пока не верят в возобновление снижения',
  'Роснефть: Выход вниз из восходящего канала омрачил перспективы',
  'ГМК Норильский Никель: Коррекция после обновления максимумов',
  'МосБиржа: Спрос на бумаги все еще низкий',
  'ГАЗПРОМ: В случае провала к 145 и дальнейшего отскока, будем наращивать длинные позиции',
  'Нужно больше дерева, милорд. Инвестиции в лес и пиломатериалы',
  'Акции Сургутнефтегаза продолжают сползать вниз',
  'Газпром. Совет директоров не захотел выкупать акции. Продаем?',
  'Золото: Стави